# Introduction to the Rewiring America GGRF HVAC Upgrade API

Rewiring America provides an API that replicates the methodology GGRF has selected for evaluating the changes in
emissions associated with various household HVAC upgrades.

This API is primarily intended to be used by our various GGRF partners.

The API entry points are designed to take the following inputs:

- the zip code of the home to be upgraded.
- an existing heating unit described in terms of:
  - what fuel it uses;
  - how much of that fuel it uses;
  - how efficient it is.
- a new heat pump described in terms of:
  - the heat pump's efficiency.

v1 of the API is based on sample calculations in a GGRF-provided spreadsheet entitled
`Priority_Project_2_Net-Zero Building GGRF Calculator 10.18.24.xlsx`. Specifically, it
replicates the calculator found on the `B1. HVAC Update` tab.

The tab is in some sense one calculator, but because some inputs are used or unused, or conditionally required based on
the values of others, we have broken out a series of API entry points. The four main entry points compute emission
changes as a result of

- Natural gas to heat pump HVAC upgrades
- Oil to heat pump HVAC upgrades
- Propane to heat pump HVAC upgrades
- Electric to heat pump HVAC upgrades

The reason we broke these out, as opposed to having a single API entry point with an argument to specify the type of
HVAC being replaced is that conditional on what type it is, we need slightly different arguments, e.g. existing
consumption in cubic feet of natural gas vs. gallons of oil or propane, that means we have to either overload arguments
or have several optional arguments. Breaking the API into four entry points hopefully makes it more clear exactly what
is required in each case.

In addition these calculator APIs, there are three utility APIs to look up the efficiency of legacy HVAC equipment. The
reason we need these is that in order for the API entry points above to do their work, they must be told what the
efficiency of the equipment being replaced is. Sometimes this is known, but often it is not. What may be known, however,
is the efficiency standard of the old equipment, which we can then use to approximate its efficiency. These three APIs
return efficiency based on the standard met by

- Propane or gas equipment
- Oil equipment
- Electric equipment

This notebook demonstrates how to call these APIs.

## Basic Imports and Setup

In [1]:
import requests
from pathlib import Path

### API Key

You will need an API key to access the GGRF API. To obtain an
API key, please visit https://rewiring.link/api-signup.

If you make a directory called `.rwapi` under your home directory and put a
file called `api_key.txt` there with just one line containing your API key,
then the code in the next cell will be able to read it. If you prefer, you
can hard-code your key in the commented out section. If you do so, please do
not check your code into a public repository.

In [2]:
# Look for our API key is a canonical local place. You can either put your API
# key in the file ~/.rwapi/api_key.txt and this code will find it. 
api_key_path = Path.home() / ".rwapi" / "api_key.txt"

if api_key_path.is_file():
    with open(api_key_path) as f:
        API_KEY = f.read().strip()
else:
    API_KEY = None

# Alternatively, you can just put it in here.
# API_KEY = "zppa_***"

In [3]:
headers = {"Authorization": f"Bearer {API_KEY}"}

In [4]:
# The base URL for the API server

BASE_URL = "https://api.rewiringamerica.org/api/v1/ggrf/hvac-upgrade"

NATURAL_GAS_TO_ELECTRIC_URL = f"{BASE_URL}/natural-gas-to-electric"
OIL_TO_ELECTRIC_URL = f"{BASE_URL}/oil-to-electric"
PROPANE_TO_ELECTRIC_URL = f"{BASE_URL}/oil-to-electric"
ELECTRIC_TO_ELECTRIC_URL = f"{BASE_URL}/electric-to-electric"

NATURAL_GAS_AND_PROPANE_EFFICIENCY_STANDARDS_URL = (
    f"{BASE_URL}/natural-gas-and-propane-efficiency-standards"
)

OIL_EFFICIENCY_STANDARDS_URL = f"{BASE_URL}/oil-efficiency-standards"

ELECTRIC_EFFICIENCY_STANDARDS_URL = f"{BASE_URL}/electric-efficiency-standards"

## First API Call

In this first call, we will explore the emissions savings resulting from an upgrade
from a natural gas powered heating system to a heat pump.

We will provide the API with:

- a zip code where the home to be upgraded is located
- the efficiency of the new unit in BTU per watt hour (more on this later in the notebook)
- the efficiency of the existing unit in BTU per watt hour (more on this later in the notebook)
- The number of cubic feet of natural gas the existing unit consumes per year

In [5]:
NATURAL_GAS_TO_ELECTRIC_URL

'https://api.rewiringamerica.org/api/v1/ggrf/hvac-upgrade/natural-gas-to-electric'

In [6]:
response = requests.get(
    url=NATURAL_GAS_TO_ELECTRIC_URL,
    headers=headers,
    params={
        "zip_code": "30306",
        "new_unit_efficiency_btu_per_watt_hour": 17.0,
        "existing_efficiency_btu_per_watt_hour": 4.0,
        "existing_consumption_cubic_feet_per_year": 70_000.0,
    },
)

In [7]:
response.status_code

200

### Results

Assuming the call succeeded with a status code of 200 (OK), we can look at the JSON response to see:

- the predicted emissions from the existing unit, in tons of $\text{CO}_2$ equivalent per year
- the predicted emissions from the new unit
- the predicted savings, which is the emissions of the existing unit minus the emissions of the new unit.
  A positive number means we are reducing emissions.

In [8]:
response.json()

{'existing_emissions_tons_co2e': 4.0,
 'new_unit_emissions_metric_tons_co2e': 2.39,
 'emissions_reduced_metric_tons_co2e': 1.61,
 'further_information': 'Please refer to our API documentation for additional information specific to this API.'}

In this examples, emissions will be reduced by 1.61 tons of $\text{CO}_2$ equivalent.

Note that along with the results, the return value also includes further information with some
details about the API and usage of it's outputs.

In [9]:
print(response.json()["further_information"])

Please refer to our API documentation for additional information specific to this API.


## Efficiency Standards

One of the arguments we had to provide in the API call we just made was `existing_efficiency_btu_per_watt_hour`,
which represents the efficiency of the unit we are planning to replace is standardized units of BTU per watt
hour. But this is not something most people are likely to know. And this information may or may not be available
on a label or in the user manual for the existing unit. So how can we get an idea of what number we should plug in?

In order to answer this question, the GGRF spreadsheet our API is based on includes a lookup table from various
efficiency standards to their efficiency in BTUs per watt hour. We have replicated that and put it in an API. It can be called as follows:

In [10]:
response = requests.get(
    url=NATURAL_GAS_AND_PROPANE_EFFICIENCY_STANDARDS_URL,
    headers=headers,
)

In [11]:
response.status_code

200

### Results

Assuming the call succeeded with a status code of 200 (OK), we can look at the JSON response to see the names
of all the kinds of gas and propane furnaces and boilers that the GGRF knows about.

In [12]:
response.json()

{'AFUE - Gas/Propane Furnace - Condensing Prior to 2015': 3.07092747,
 'AFUE - Gas/Propane Furnace - Natural Draft (All Years)': 2.661470474,
 'AFUE - Gas/Propane Furnace - Induced Draft (All Years)': 2.797956139,
 'AFUE - Gas/Propane Furnace - North': 3.241534551,
 'AFUE - Gas/Propane Furnace - South': 3.07092747,
 'AFUE - Gas/Propane Boiler - ENERGY STAR after Oct 2014': 3.07092747,
 'AFUE - Gas/Propane Boiler - ENERGY STAR after Oct 2004': 2.900320388,
 'AFUE - Gas/Propane Boiler - Condensing': 3.07092747,
 'AFUE - Gas/Propane Boiler - Induced Draft': 2.797956139,
 'AFUE - Gas/Propane Boiler - Natural Draft': 2.661470474}

So now, if we know the unit that we are replacing is a ENERGY STAR gas boiler installed in 2006,
we could look at these values and see that we should use the number `2.900320388` corresponding to
`'AFUE - Gas/Propane Boiler - ENERGY STAR after Oct 2004'`. If it were from 2015, on the other hand,
we would use the value from `'AFUE - Gas/Propane Boiler - ENERGY STAR after Oct 2014'`.

In [13]:
existing_efficiency_btu_per_watt_hour = response.json()[
    "AFUE - Gas/Propane Boiler - ENERGY STAR after Oct 2004"
]
existing_efficiency_btu_per_watt_hour

2.900320388

### Call the upgrade API again

Now we can make our original call again with this number.

In [14]:
response = requests.get(
    url=NATURAL_GAS_TO_ELECTRIC_URL,
    headers=headers,
    params={
        "zip_code": "30306",
        "new_unit_efficiency_btu_per_watt_hour": 17.0,
        "existing_efficiency_btu_per_watt_hour": existing_efficiency_btu_per_watt_hour,
        "existing_consumption_cubic_feet_per_year": 70_000.0,
    },
)

In [15]:
response.status_code

200

In [16]:
response.json()

{'existing_emissions_tons_co2e': 4.0,
 'new_unit_emissions_metric_tons_co2e': 1.25,
 'emissions_reduced_metric_tons_co2e': 2.74,
 'further_information': 'Please refer to our API documentation for additional information specific to this API.'}

The numbers are a little different this time, as expected, because we changed the efficiency of the
existing unit.

## Upgrading from Oil

Our past examples involved upgrades from natural gas to a heat pump. The next example will show how similar
the API is for upgrades from oil. The main difference is that instead of passing in the current annual fuel consumption in cubic feet of gas, we pass it in in terms of gallons of oil.

We will start by looking up the efficiency of an existing oil furnace whose efficiency standard is known.

In [17]:
response = requests.get(
    url=OIL_EFFICIENCY_STANDARDS_URL,
    headers=headers,
)

In [18]:
response.status_code

200

In [19]:
response.json()

{'AFUE - Oil Furnace - Condensing': 3.07092747,
 'AFUE - Oil Furnace - ENERGY STAR after Feb 2003': 2.900320388,
 'AFUE - Oil Furnace - Retention Head Burner': 2.729713306,
 'AFUE - Oil Furnace - Conventional Burner': 2.456741976,
 'AFUE - Oil Boiler - ENERGY STAR after Oct 2014': 2.968563221,
 'AFUE - Oil Boiler - ENERGY STAR after April 2002': 2.900320388,
 'AFUE - Oil Boiler - After Sept. 2012 (All except ES)': 2.866198972,
 'AFUE - Oil Boiler - Induced Draft': 2.797956139,
 'AFUE - Oil Boiler - Natural Draft': 2.661470474}

In [20]:
existing_efficiency_btu_per_watt_hour = response.json()[
    "AFUE - Oil Boiler - Natural Draft"
]
existing_efficiency_btu_per_watt_hour

2.661470474

Now we will make the call to see what the upgrade will change, using the efficiency of the existing
unit that we just looked up.

In [21]:
response = requests.get(
    url=OIL_TO_ELECTRIC_URL,
    headers=headers,
    params={
        "zip_code": "02906",
        "new_unit_efficiency_btu_per_watt_hour": 16.0,
        "existing_efficiency_btu_per_watt_hour": existing_efficiency_btu_per_watt_hour,
        "existing_consumption_gallons_per_year": 1000.0,
    },
)

In [22]:
response.status_code

200

In [23]:
response.json()

{'existing_emissions_tons_co2e': 12.02,
 'new_unit_emissions_metric_tons_co2e': 1.29,
 'emissions_reduced_metric_tons_co2e': 10.73,
 'further_information': 'Please refer to our API documentation for additional information specific to this API.'}

## Upgrading from Propane

This is very similar to upgrading from natural gas. We query the efficiency standards and then
call the API.

In [24]:
response = requests.get(
    url=NATURAL_GAS_AND_PROPANE_EFFICIENCY_STANDARDS_URL,
    headers=headers,
)

In [25]:
response.status_code

200

In [26]:
response.json()

{'AFUE - Gas/Propane Furnace - Condensing Prior to 2015': 3.07092747,
 'AFUE - Gas/Propane Furnace - Natural Draft (All Years)': 2.661470474,
 'AFUE - Gas/Propane Furnace - Induced Draft (All Years)': 2.797956139,
 'AFUE - Gas/Propane Furnace - North': 3.241534551,
 'AFUE - Gas/Propane Furnace - South': 3.07092747,
 'AFUE - Gas/Propane Boiler - ENERGY STAR after Oct 2014': 3.07092747,
 'AFUE - Gas/Propane Boiler - ENERGY STAR after Oct 2004': 2.900320388,
 'AFUE - Gas/Propane Boiler - Condensing': 3.07092747,
 'AFUE - Gas/Propane Boiler - Induced Draft': 2.797956139,
 'AFUE - Gas/Propane Boiler - Natural Draft': 2.661470474}

In [27]:
existing_efficiency_btu_per_watt_hour = response.json()[
    "AFUE - Gas/Propane Boiler - ENERGY STAR after Oct 2004"
]

In [28]:
response = requests.get(
    url=PROPANE_TO_ELECTRIC_URL,
    headers=headers,
    params={
        "zip_code": "30306",
        "new_unit_efficiency_btu_per_watt_hour": 17.0,
        "existing_efficiency_btu_per_watt_hour": existing_efficiency_btu_per_watt_hour,
        "existing_consumption_gallons_per_year": 1_000.0,
    },
)

In [29]:
response.status_code

200

In [30]:
response.json()

{'existing_emissions_tons_co2e': 12.02,
 'new_unit_emissions_metric_tons_co2e': 2.4,
 'emissions_reduced_metric_tons_co2e': 9.62,
 'further_information': 'Please refer to our API documentation for additional information specific to this API.'}

## Upgrading from Electric

We also have an API for upgrading from electric heat, such as baseboard
heating, to a heat pump. The usage pattern is the same as for the other 
fuels above.

In [31]:
response = requests.get(
    url=ELECTRIC_EFFICIENCY_STANDARDS_URL,
    headers=headers,
)

In [32]:
response.status_code

200

In [33]:
response.json()

{'SEER - ENERGY STAR 2006 thru Aug 2015': 14.5,
 'SEER - 2015 or newer (South)': 14.0,
 'SEER - 2015 or newer (North)': 13.0,
 'SEER - 2006 to 2014': 13.0,
 'SEER - 1992 through 2005': 10.0,
 'SEER - Prior to 1992': 9.0,
 'SEER - Mini-split / Ductless (All years)': 19.0,
 'HSPF - ENERGY STAR Sept 2015 to present': 8.5,
 'HSPF - ENERGY STAR 2006 thru Aug 2015': 8.2,
 'HSPF - 2015 or newer': 8.2,
 'HSPF - 2006 to 2014': 7.7,
 'HSPF - 1992 through 2005': 7.1,
 'HSPF - Prior to 1992': 6.6,
 'HSPF - Mini-split / Ductless (All years)': 10.0,
 'EER - Window A/C': 9.0}

In [34]:
existing_efficiency_btu_per_watt_hour = response.json()["SEER - 1992 through 2005"]

In [35]:
# In this API we express current HVAC usage in kWh instead of gallons or
# cubic feet of fuel consumed.

response = requests.get(
    url=ELECTRIC_TO_ELECTRIC_URL,
    headers=headers,
    params={
        "zip_code": "30306",
        "new_unit_efficiency_btu_per_watt_hour": 17.0,
        "existing_efficiency_btu_per_watt_hour": existing_efficiency_btu_per_watt_hour,
        "annual_hvac_usage_kwh": 2_000.0,
    },
)

In [36]:
response.status_code

200

In [37]:
response.json()

{'existing_emissions_tons_co2e': 0.8142031746031746,
 'new_unit_emissions_metric_tons_co2e': 0.4789430438842203,
 'emissions_reduced_metric_tons_co2e': 0.3352601307189542,
 'further_information': 'Please refer to our API documentation for additional information specific to this API.'}